# Zpracování a prezentace dat

Notebook pro získání moderátorů a respondentů zastoupených ve vybraných pořadech vysílání Českého rozhlasu.  

## Knihovny

In [ ]:
import csv
import math
import os

import pandas as pd
import requests
import cro.schedule as schedule
from pathlib import Path


DATA = Path("./data")

# import sys
#!{sys.executable} -m pip install scrapy
# import sys; sys.path
# import numpy; numpy.__path__

## Konfigurace

Od uživatele chceme aby zadal požadované období (dny) pro které budeme provádět analýzu.

### Vstup od uživatele

In [ ]:
dates = [f"2023-05-{i}" for i in range(1, 31)]

print(f"Working with dates: {dates}.")

## Zpracování

### Získej program pro stanici Plus a Radiožurnál

In [ ]:
client = schedule.Client("plus")
schedules_ps = list(client.get_day_schedule(date) for date in dates)

In [ ]:
client = schedule.Client("radiozurnal")
schedules_rz = list(client.get_day_schedule(date) for date in dates)

In [ ]:
schedules = schedules_rz + schedules_ps

for s in schedules:
    print(s)

### Vyber jen požadované pořady

In [ ]:
schedule_data = []

for schedule in schedules:
    for index, show in enumerate(schedule.shows):
        if len(show.moderators) == 0:
            moderators = ""
        else:
            moderators = ";".join(p.name for p in show.moderators)
        relation_type = "repríza" if show.repetition else "premiéra"

        line = dict(
            station_name=show.station.name,
            show_type=show.type,
            show_kind_name=show.kind.name,
            show_description=show.description,
            show_id=show.id,
            show_title=show.title,
            show_date=show.date,
            show_since=show.since,
            show_till=show.till,
            relation_type=relation_type,
            show_duration_minutes=math.ceil(show.duration / 60),
            moderators=moderators,
        )

        schedule_data.append(line)

### Ulož program do CSV pro další zpracování

In [132]:
# Uložíme CSV pro další zpracování.
with open(DATA / "schedule.csv", mode="w") as file:
    writer = csv.DictWriter(file, fieldnames=schedule_data[0].keys())
    writer.writeheader()
    writer.writerows(schedule_data)

---
**ZAČNI ZDE, POKUD CHCEŠ PRACOVAT S JIŽ ULOŽENÝM PROGRAMEM**

In [133]:
SHOWS_WANTED = (
    "Pro a proti",
    "Interview Plus",
    "Rozhovory a komentáře",
    "Dvacet minut Radiožurnálu",
    "Hlavní zprávy - rozhovory a komentáře",
    "Speciál",
)

### Načti program a získej premiéry

- Načteme data do tabulky a aplikujeme voodoo a černou magii.
  - Vybereme pouze premiéry pořadů.
- Respondenty získáme pomocí parsování a služby Geneea (*entity recognition*).


In [134]:
df = pd.read_csv(DATA / "schedule.csv")
df = df[df.relation_type == "premiéra"]  # FIXME

### Ukaž unikátní názvy pořadů 

In [135]:
shows_ps = set(df[df["station_name"] == "Plus"]["show_title"].to_list())
shows_rz = set(df[df["station_name"] == "Radiožurnál"]["show_title"].to_list())

print("POŘADY STANICE PLUS:\n", shows_ps)
print("POŘADY STANICE RADIOŽURNÁL:\n", shows_rz)

POŘADY STANICE PLUS:
 {'Mezi námi', 'Peníze a vliv', 'Zprávy v angličtině/News in English', 'Svět ve 20 minutách', 'Sváteční Plus', 'Hovory', 'Laboratoř', 'RozhlaSTO', 'Vertikála', 'Bruselské chlebíčky', 'Ex libris', 'Kultura Plus', 'Život k nezaplacení - speciál', 'Hlavní zprávy', 'Studio Leonardo', 'Zprávy v půl', 'Jak to bylo doopravdy', 'Ranní Plus', 'Online Plus', 'Týden Plus', 'Chyba systému', 'Interview Plus', 'Dvacet minut Radiožurnálu', 'Za 5 minut 12 - speciál o vládních reformách', 'Jak to vidí...', 'Řečí peněz', 'Historie Plus', 'Hlavní zprávy - rozhovory, komentáře', 'Vinohradská 12', 'Pro a proti', 'Portréty', 'Radiokniha', 'Natura', 'Reportáže zahraničních zpravodajů', 'Dokument', 'Odpolední Plus', 'Život k nezaplacení', 'Archiv Plus', 'Dvacítka Radiožurnálu', 'Dvacet minut Radiožurnálu - speciál', 'Téma Plus', 'Dokument Plus', '17. 06 Dvacet minut Radiožurnálu', 'Souvislosti Plus', 'Osobnost Plus', 'Zprávy', 'Leonardo Plus', 'Zaostřeno', 'Názory a argumenty', 'Za 5 minu

In [136]:
# Vyber pouze sledované pořady
df = df[df["show_title"].isin(SHOWS_WANTED)]  # FIXME
df.head(100)

,station_name,show_type,show_kind_name,show_description,show_id,show_title,show_date,show_since,show_till,relation_type,show_duration_minutes,moderators
44,Radiožurnál,AFTERNOON,Zpravodajství,"O všem, co se právě děje",18701135,Hlavní zprávy - rozhovory a komentáře,2023-05-01,12:10:00,12:30:00,premiéra,20,NaN
61,Radiožurnál,AFTERNOON,Publicistika,Hosty jsou historik Michal Stehlík a publicist...,18701173,Dvacet minut Radiožurnálu,2023-05-01,17:06:00,17:30:00,premiéra,24,Vladimír Kroc
129,Radiožurnál,AFTERNOON,Zpravodajství,"O všem, co se právě děje",18701333,Hlavní zprávy - rozhovory a komentáře,2023-05-02,12:10:00,12:30:00,premiéra,20,NaN
147,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Markéta Gregorová z Pirátské strany,...",18701371,Dvacet minut Radiožurnálu,2023-05-02,17:06:00,17:30:00,premiéra,24,Vladimír Kroc
150,Radiožurnál,EVENING,Zpravodajství,"O všem, co se právě děje",18701378,Hlavní zprávy - rozhovory a komentáře,2023-05-02,18:10:59,18:30:00,premiéra,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3779,Plus,NOON,Publicistika,"Jindřich Šídlo, novinář. Pietního aktu k uctěn...",18672005,Interview Plus,2023-05-15,11:34:00,12:00:00,premiéra,26,Šárka Fenyková
3800,Plus,AFTERNOON,Publicistika,"Hostem je Štěpán Macháček, zvláštní zpravodaj ...",18672036,Dvacet minut Radiožurnálu,2023-05-15,17:06:00,17:30:00,premiéra,24,NaN
3858,Plus,MORNING,Publicistika,Vinaři ještě nemají ohledně nezdanění vína vyh...,18672105,Pro a proti,2023-05-16,09:34:00,10:00:00,premiéra,26,Karolína Koubová
3866,Plus,NOON,Publicistika,"Libor Matoušek, předseda představenstva Asocia...",18672117,Interview Plus,2023-05-16,11:34:00,11:59:00,premiéra,25,Šárka Fenyková


## Kontrola

MUSÍME AUTOMATIZOVAT -- TOTO JE ZASTARALÉ A MANUÁLNÍ!

Zkontrolujeme si, jestli máme očekávaný počet premiér pořadů (nyní za měsíc):
- Dvacet minut Radiožurnálu: 23 (premiér/březen) * 2 (stanic) = 46 premiér / březen (23 na Radiožurnálu)
- Interview Plus: 23 (premiér/březen) * 1 (stanic) = 23 premiér / březen
- Pro a proti: 23 (premiér/březen) * 1 (stanic) = 23 premiér / březen
- Hlavní zprávy - rozhovory a komentáře: 23 * 2 (dvakrát za prac. den) * 2 (stanic) = 92 (46 na Radiožurnálu)


In [137]:
# Kolik máme řádků/premiér pořadů (epizod).
len(df)

131

In [138]:
#  Na Plusu se premiéry objevují jen pro dva dny v týdnu, zřejmě můžeme Plus úplně vyfiltrovat.
df = df[
    ~(
        (df["show_title"] == "Dvacet minut Radiožurnálu")
        & (df["station_name"] == "Plus")
    )
]
df = df[
    ~(
        (df["show_title"] == "Hlavní zprávy - rozhovory a komentáře")
        & (df["station_name"] == "Plus")
    )
]

print(
    "Očekávaný počet premiér 'Dvacet minut Radiožurnálu' odpovídá:",
    len(df[df["show_title"] == "Dvacet minut Radiožurnálu"]) == 23,
)
print(
    "Očekávaný počet premiér 'Interview Plus' odpovídá:",
    len(df[df["show_title"] == "Interview Plus"]) == 23,
)
print(
    "Očekávaný počet premiér 'Pro a proti' odpovídá:",
    len(df[df["show_title"] == "Pro a proti"]) == 23,
)
print(
    "Očekávaný počet premiér 'Hlavní zprávy - rozhovory a komentáře' odpovídá:",
    len(df[df["show_title"] == "Hlavní zprávy - rozhovory a komentáře"]) == 46,
    len(df[df["show_title"] == "Hlavní zprávy - rozhovory a komentáře"]),
)

# df[df["show_title"] ==  "Hlavní zprávy - rozhovory a komentáře" ].head(100)

Očekávaný počet premiér 'Dvacet minut Radiožurnálu' odpovídá: False
Očekávaný počet premiér 'Interview Plus' odpovídá: False
Očekávaný počet premiér 'Pro a proti' odpovídá: True
Očekávaný počet premiér 'Hlavní zprávy - rozhovory a komentáře' odpovídá: False 47


### Získej popis epizod pro NLP analýzu

In [139]:
index_description_df = df[["show_title", "show_description"]]

#### Ukázka záznamů

In [140]:
index_description_df.head(5)

,show_title,show_description
44,Hlavní zprávy - rozhovory a komentáře,"O všem, co se právě děje"
61,Dvacet minut Radiožurnálu,Hosty jsou historik Michal Stehlík a publicist...
129,Hlavní zprávy - rozhovory a komentáře,"O všem, co se právě děje"
147,Dvacet minut Radiožurnálu,"Hostem je Markéta Gregorová z Pirátské strany,..."
150,Hlavní zprávy - rozhovory a komentáře,"O všem, co se právě děje"


#### Počet záznamů

In [141]:
len(index_description_df)

111

### Vytvoř objekty pro NLP analýzu

Po potřeby Geneea Media V2 API vytvoříme JSON objekty, které zasíláme v dotazu. Jako index (atribut `id`) použijeme index řádku datového rámce, ten pak dostaneme po zpracování zpět. Tím můžeme získané entity opět spojit s původními řádky datového rámce. Pro jeden analyzovaný text můžeme dostat 0 až N rozpoznaných osob. Nekteré z nich mohou být moderátoři, ty se snažíme odfiltrovat pomocí daného seznamu.

In [144]:
inputs = []
from uuid import uuid4
for _, record in index_description_df.iterrows():
    inputs.append(
        {
            "id": str(uuid4()), # Unique ID so Geneea can better search a request log.
            "paraSpecs": [{"type": "text", "text": record.show_description}],
        }
    )

In [145]:
BASE_URL = "https://media-api.geneea.com/v2/"
HEADERS = {"content-type": "application/json", "X-API-Key": os.getenv("GENEEA_API_KEY")}

results = [
    requests.post(f"{BASE_URL}nlp/analyze", json=input, headers=HEADERS).json()
    for input in inputs
]

In [146]:
len(results) == len(index_description_df)

True

### Uložíme  získané entity/osoby spolu s původním indexem záznamu 

Rozpoznaná entita má buď typ `person` nebo v některých případech typ `other`. To pak musíme hledat klíč `feats` a v něm dále `detectedType`. 

**Klíče nemusí existovat, proto testujeme jejich přítomnost!**

In [198]:
index_org = []
index_person = []
for result in results:
    if "entities" in result:
        for entity in result["entities"]:
            if entity["type"] == "person":
                index_person.append((result["id"], entity["stdForm"], entity["gkbId"]))
            if "feats" in entity:
                if "detectedType" == entity["feats"]:
                    if entity["feats"]["detectedType"] == "person":
                        index_person.append(
                            (result["id"], entity["stdForm"], entity["gkbId"])
                        )
    if "entities" in result:
        for entity in result["entities"]:
            if entity["type"] == "organization":
                index_org.append((result["id"], entity["stdForm"], entity["gkbId"]))
            if "feats" in entity:
                if "detectedType" == entity["feats"]:
                    if entity["feats"]["detectedType"] == "organization":
                        index_org.append(
                            (result["id"], entity["stdForm"], entity["gkbId"])
                        )

In [200]:
index_org

org_df = pd.DataFrame(index_org, columns=["index", "organization", "gkbid"])
org_df.set_index("index")
org_df.index.astype(int)
org_df

,index,organization,gkbid
0,870bcf9f-6709-4f8f-aba9-a190f9f0557e,Česká pirátská strana,G341085
1,870bcf9f-6709-4f8f-aba9-a190f9f0557e,Evropský parlament,G8889
2,870bcf9f-6709-4f8f-aba9-a190f9f0557e,Evropská komise,G8880
3,8e7c048f-a0d5-4b4c-9999-754cc52aea6a,KDU-ČSL,G1142687
4,a3ace090-8e1b-4bda-bf31-6e8e795603eb,Finanční správa,G14806057
...,...,...,...
70,cdb6c90d-b6e0-46dc-a4f0-d0d725d42eb9,Evropský parlament,G8889
71,e2643d60-2763-4a25-a5ba-5cdaa1f6c047,Česká národní banka,G251062
72,946108a5-66f8-4421-8362-3e07f423191d,Evropská unie,G458
73,946108a5-66f8-4421-8362-3e07f423191d,ODS,G828099


In [148]:
respondents_df = pd.DataFrame(index_person, columns=["index", "respondent", "gkbid"])
respondents_df.set_index("index")
respondents_df.index.astype(int)
respondents_df

,index,respondent,gkbid
0,2d7ff747-c042-4032-b83c-e38b7776ffba,Michal Stehlík,G12036808
1,2d7ff747-c042-4032-b83c-e38b7776ffba,Vladimír Kroc,G17145442
2,870bcf9f-6709-4f8f-aba9-a190f9f0557e,Markéta Gregorová,G60552203
3,870bcf9f-6709-4f8f-aba9-a190f9f0557e,Ursula von der Leyenová,G60772
4,870bcf9f-6709-4f8f-aba9-a190f9f0557e,Petr Pavel,G2080040
...,...,...,...
123,946108a5-66f8-4421-8362-3e07f423191d,Emmanuel Macron,G3052772
124,946108a5-66f8-4421-8362-3e07f423191d,Olaf Scholz,G61053
125,946108a5-66f8-4421-8362-3e07f423191d,Recep Tayyip Erdoğan,G39259
126,946108a5-66f8-4421-8362-3e07f423191d,Alexandr Vondra,G939484


In [87]:
# Vrátíme osoby do původního dataframu. Grupujeme si osoby podle indexu.
merged_respondents_df = respondents_df.groupby(["index"])
merged_respondents_df = merged_respondents_df["respondent"].apply(";".join)
merged_respondents_df.reset_index().set_index("index")
merged_respondents_df = pd.DataFrame(merged_respondents_df)
merged_respondents_df

,respondent
index,
1013,Jana Klímová
1261,Štěpán Macháček;Recep Tayyip Erdoğan
1347,Mikuláš Bek
147,Markéta Gregorová;Ursula von der Leyenová;Petr...
1526,Marian Jurečka
1601,Karel Havlíček;Petr Fiala
1848,Petr Fiala;Vladimír Kroc
2017,Daniela Kovářová
2187,Vladimír Kroc


In [149]:
merged_respondents_df.index = merged_respondents_df.index.astype(int)
print(merged_respondents_df.index.dtype)

# Iterace: Tohle je pěkná blbost z hlediska rychlosti, ale merge nějak nefungoval.
# Nicméně smyčka funguje a zjistili jsme, že indexy neměli správny typ, proto původní merge nefungoval.
# nechávám jako důkaz a připomínku vlastní blbosti.
# for row in df.iterrows():
#     if row[0] in merged_respondents_df.index:
#         print(row[0], merged_respondents_df.loc[row[0]])

int32


In [167]:
final_df = df.join(merged_respondents_df)

In [168]:
final_df = final_df.assign(respondent=final_df.respondent.str.split(";")).explode("respondent")

In [169]:
backup = final_df

In [182]:
final_df = backup

In [183]:
final_df = final_df.assign(moderators=final_df.moderators.str.split(";")).explode("moderators")

In [185]:
# Remove respondents which was identified as moderators. Also removes NaNs.
final_df = final_df[final_df.respondent != final_df.moderators]

In [186]:
final_df = final_df[(~final_df.respondent.isna()) & (~final_df.moderators.isna())]

In [187]:
final_df

,station_name,show_type,show_kind_name,show_description,show_id,show_title,show_date,show_since,show_till,relation_type,show_duration_minutes,moderators,respondent
61,Radiožurnál,AFTERNOON,Publicistika,Hosty jsou historik Michal Stehlík a publicist...,18701173,Dvacet minut Radiožurnálu,2023-05-01,17:06:00,17:30:00,premiéra,24,Vladimír Kroc,Michal Stehlík
147,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Markéta Gregorová z Pirátské strany,...",18701371,Dvacet minut Radiožurnálu,2023-05-02,17:06:00,17:30:00,premiéra,24,Vladimír Kroc,Markéta Gregorová
147,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Markéta Gregorová z Pirátské strany,...",18701371,Dvacet minut Radiožurnálu,2023-05-02,17:06:00,17:30:00,premiéra,24,Vladimír Kroc,Ursula von der Leyenová
147,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Markéta Gregorová z Pirátské strany,...",18701371,Dvacet minut Radiožurnálu,2023-05-02,17:06:00,17:30:00,premiéra,24,Vladimír Kroc,Petr Pavel
235,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Marek Výborný, předseda poslaneckého...",18701570,Dvacet minut Radiožurnálu,2023-05-03,17:05:04,17:29:17,premiéra,25,Vladimír Kroc,Marek Výborný
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,Plus,AFTERNOON,Publicistika,Francouzský prezident Emmanuel Macron a německ...,18673607,Pro a proti,2023-05-30,14:33:00,15:00:00,premiéra,27,Lukáš Matoška,Emmanuel Macron
5083,Plus,AFTERNOON,Publicistika,Francouzský prezident Emmanuel Macron a německ...,18673607,Pro a proti,2023-05-30,14:33:00,15:00:00,premiéra,27,Lukáš Matoška,Olaf Scholz
5083,Plus,AFTERNOON,Publicistika,Francouzský prezident Emmanuel Macron a německ...,18673607,Pro a proti,2023-05-30,14:33:00,15:00:00,premiéra,27,Lukáš Matoška,Recep Tayyip Erdoğan
5083,Plus,AFTERNOON,Publicistika,Francouzský prezident Emmanuel Macron a německ...,18673607,Pro a proti,2023-05-30,14:33:00,15:00:00,premiéra,27,Lukáš Matoška,Alexandr Vondra


In [191]:
len(final_df[final_df.moderators == final_df.respondent]) == 0

True

In [192]:
final_df.to_csv(DATA / "final_schedule.csv")

In [195]:
final_df

,station_name,show_type,show_kind_name,show_description,show_id,show_title,show_date,show_since,show_till,relation_type,show_duration_minutes,moderators,respondent
61,Radiožurnál,AFTERNOON,Publicistika,Hosty jsou historik Michal Stehlík a publicist...,18701173,Dvacet minut Radiožurnálu,2023-05-01,17:06:00,17:30:00,premiéra,24,Vladimír Kroc,Michal Stehlík
147,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Markéta Gregorová z Pirátské strany,...",18701371,Dvacet minut Radiožurnálu,2023-05-02,17:06:00,17:30:00,premiéra,24,Vladimír Kroc,Markéta Gregorová
147,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Markéta Gregorová z Pirátské strany,...",18701371,Dvacet minut Radiožurnálu,2023-05-02,17:06:00,17:30:00,premiéra,24,Vladimír Kroc,Ursula von der Leyenová
147,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Markéta Gregorová z Pirátské strany,...",18701371,Dvacet minut Radiožurnálu,2023-05-02,17:06:00,17:30:00,premiéra,24,Vladimír Kroc,Petr Pavel
235,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Marek Výborný, předseda poslaneckého...",18701570,Dvacet minut Radiožurnálu,2023-05-03,17:05:04,17:29:17,premiéra,25,Vladimír Kroc,Marek Výborný
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,Plus,AFTERNOON,Publicistika,Francouzský prezident Emmanuel Macron a německ...,18673607,Pro a proti,2023-05-30,14:33:00,15:00:00,premiéra,27,Lukáš Matoška,Emmanuel Macron
5083,Plus,AFTERNOON,Publicistika,Francouzský prezident Emmanuel Macron a německ...,18673607,Pro a proti,2023-05-30,14:33:00,15:00:00,premiéra,27,Lukáš Matoška,Olaf Scholz
5083,Plus,AFTERNOON,Publicistika,Francouzský prezident Emmanuel Macron a německ...,18673607,Pro a proti,2023-05-30,14:33:00,15:00:00,premiéra,27,Lukáš Matoška,Recep Tayyip Erdoğan
5083,Plus,AFTERNOON,Publicistika,Francouzský prezident Emmanuel Macron a německ...,18673607,Pro a proti,2023-05-30,14:33:00,15:00:00,premiéra,27,Lukáš Matoška,Alexandr Vondra


In [117]:
r = final_df.apply(lambda row: list(set([row['respondent']]).difference(set(row['moderators']))), axis=1)

### Zjisti funkce osob pomocí Geneea

**Nyní nemám přístup k této funkcionalitě.**

Zjistíme funkce pro všechny rozpoznané osoby. Prozatím negunguje / access denied

In [ ]:
# ids = respondents_df["gkbid"].to_list()
# data_input = {
#     "ids": ids,
#     "language": "cs",
# }

# result = requests.post(
#     f"{BASE_URL}knowledgebase/infoboxes", json=data_input, headers=HEADERS
# )
# result.content

# ČÁST 2: Zpracování dat z Go programu

In [201]:
example = {"show":"Hlavní zprávy - rozhovory a komentáře","episode":"Ústavní soudci. Ukrajinská ofenziva. Sdílení elektřiny","date":"2023-06-05","time":"12:10","link":"https://plus.rozhlas.cz/poledni-publicistika-ustavni-soudci-ukrajinska-ofenziva-sdileni-elektriny-9007443","teaser":"V jaké kondici je současný Ústavní soud, jehož složení se tento rok zásadně obměňuje? Začala ukrajinská ofenzíva?  Co chybí k tomu, aby mohlo začít v praxi sdílení elektřiny z obnovitelných zdrojů energie?","moderator":{"firstName":"Věra","lastName":"Štechrová"},"guests":[{"firstName":"Ondřej","lastName":"Preuss","function":"z Katedry ústavního práva při Právnické fakultě Univerzity Karlovy"},{"firstName":"Pavel","lastName":"Macko","function":"generálporučík v záloze"},{"firstName":"Štěpán","lastName":"Chalupa","function":"předseda Komory obnovitelných zdrojů energie"}],"topicsCount":3}

for k, v in example.items():
    print(k,v)


show Hlavní zprávy - rozhovory a komentáře
episode Ústavní soudci. Ukrajinská ofenziva. Sdílení elektřiny
date 2023-06-05
time 12:10
link https://plus.rozhlas.cz/poledni-publicistika-ustavni-soudci-ukrajinska-ofenziva-sdileni-elektriny-9007443
teaser V jaké kondici je současný Ústavní soud, jehož složení se tento rok zásadně obměňuje? Začala ukrajinská ofenzíva?  Co chybí k tomu, aby mohlo začít v praxi sdílení elektřiny z obnovitelných zdrojů energie?
moderator {'firstName': 'Věra', 'lastName': 'Štechrová'}
guests [{'firstName': 'Ondřej', 'lastName': 'Preuss', 'function': 'z Katedry ústavního práva při Právnické fakultě Univerzity Karlovy'}, {'firstName': 'Pavel', 'lastName': 'Macko', 'function': 'generálporučík v záloze'}, {'firstName': 'Štěpán', 'lastName': 'Chalupa', 'function': 'předseda Komory obnovitelných zdrojů energie'}]
topicsCount 3


In [203]:
import json
with open("./data/rozhovory.data") as f:
    lines = f"[{','.join(f.readlines())}]"
    rows = json.loads(lines)

In [230]:

rdf = pd.DataFrame(rows)

In [231]:
rdf.head()

,show,episode,date,time,link,teaser,moderator,guests,topicsCount
0,Hlavní zprávy - rozhovory a komentáře,Ústavní soudci. Ukrajinská ofenziva. Sdílení e...,2023-06-05,12:10,https://plus.rozhlas.cz/poledni-publicistika-u...,"V jaké kondici je současný Ústavní soud, jehož...","{'firstName': 'Věra', 'lastName': 'Štechrová'}","[{'firstName': 'Ondřej', 'lastName': 'Preuss',...",3
1,Hlavní zprávy - rozhovory a komentáře,Duel Fialy a Babiše. Nedostatek pediatrů. Demo...,2023-06-04,12:10,https://plus.rozhlas.cz/poledni-publicistika-d...,"Jak vypadala debata na TV Prima, kde se poprvé...","{'firstName': 'Tomáš', 'lastName': 'Pancíř'}","[{'firstName': 'Jan', 'lastName': 'Kubáček', '...",4
2,Hlavní zprávy - rozhovory a komentáře,Ruské pozemky v Česku. Festival romské kultury...,2023-06-03,12:10,https://plus.rozhlas.cz/poledni-publicistika-r...,"Jak složité bude pro české úřady, aby přiměly ...","{'firstName': 'Tomáš', 'lastName': 'Pancíř'}","[{'firstName': 'Tomáš', 'lastName': 'Petříček'...",3
3,Hlavní zprávy - rozhovory a komentáře,Ukrajinská protiofenziva. Úsporná opatření. Mi...,2023-06-02,18:10,https://plus.rozhlas.cz/odpoledni-publicistika...,Mohou být nejnovější vzdušné útoky z obou stra...,"{'firstName': 'Tomáš', 'lastName': 'Pancíř'}","[{'firstName': 'Vojtěch', 'lastName': 'Boháč',...",4
4,Hlavní zprávy - rozhovory a komentáře,Poplatky za komunální odpad. Nové podmínky pro...,2023-06-02,12:10,https://plus.rozhlas.cz/poledni-publicistika-p...,Jak citelně by se obyvatel měst a obcí mohl do...,"{'firstName': 'Tomáš', 'lastName': 'Pancíř'}","[{'firstName': 'Oldřich', 'lastName': 'Vávra',...",3


In [232]:

rdf['moderator'] = rdf.moderator.apply(lambda x: f"{x['lastName']},{x['firstName']}")


In [239]:
rdf = rdf.explode('guests')


In [240]:
rdf["respondent_first_name"] = rdf.guests.apply(lambda x: f"{x['firstName']}")
rdf["respondent_last_name"] = rdf.guests.apply(lambda x: f"{x['lastName']}")

In [247]:
rdf = rdf.drop('guests', axis=1)

In [248]:
rdf.to_csv("data/final_rozhovory.csv")